In [1]:
#IMPORT SEMUA LIBARARY

In [2]:
#LIBARARY PANDAS
import pandas as pd
#LIBARARY KONEKSI KE POSTGRE
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
#LIBARARY UNTUK BASEPATH
import os

In [3]:
#FUNGSI UNTUK UPLOAD FILE CSV KE POSTGRESQL

In [24]:
def uploadToPSQL(columns, table, filePath, engine):
    #MENDAPATKAN FILE DARI CSV, DISINI UNTUK HEADERNYA DISKIP DAN BAHASANYA MENGGUNAKAN UTF-8
    df = pd.read_csv(
        os.path.abspath(filePath),
        names=columns,
        skiprows=1, 
        header=None,
        keep_default_na=False,
        encoding='utf-8'
    )
    
    #MEMFILTER DATA YANG KOSONG
    df.fillna('')
    #MENGAMBIL VALUE DARI ARRAY DAN DIJADIKAN VARIABLE
    newsid= df['news_id']
    newscode= df['news_code']
    newslink= df['news_link']
    newsdate= df['news_date']
    newstitle= df['news_title']
    #VALUE TAG HTML DIREMOVE
    newscontent = df['news_content'].str.replace('<[^<]+?>', ' ')
    #DATA YANG DICARI DIMASUKAN DISINI, DAPAT MENCARI LEBIH DARI 1 DATA
    arrayfind = ['cukai', 'pemerintahan', 'kementerian']
    wilayah = df['media_displayName']
    strfind = ', '.join(arrayfind)
    arraycontent = []
    #DATA DIATAS DIJADIKAN DALAM SATU ARRAY
    datafix = [list(list(x) for x in zip(newsid, newscode, newslink, newsdate, newstitle, newscontent, wilayah))]
    
    #LOOPING ATAU PERULANGAN DATA DIATAS
    for newscontent in datafix[0]:
        #DATA HTML DISINI DIPECAH BERDASARKAN TITIK
        x = newscontent[5].split('.')
        #PENCARIAN DATA
        matching = [s for s in x if any(xs in s for xs in arrayfind)]
        #MENCOCOKAN DATA, JIKA COCOK DATA AKAN DITAMPUNG DIVARIABLE LAGI
        if matching != []:
            if len(matching) > 1:
                for matcharray in matching:
                    arraycontent.append(tuple([matcharray, newscontent[0], newscontent[1], newscontent[2], newscontent[3], newscontent[4], strfind, newscontent[6]]))
            else:
                arraycontent.append(tuple([str(matching)[2:-2], newscontent[0], newscontent[1], newscontent[2], newscontent[3], newscontent[4], strfind, newscontent[6]]))
    rawstr = str(arraycontent)[1:-1]
    
    #KONEKSI KE DATABASE
    try:
        #PENGECEKAN KONEKSI DATABASE
        connection = psycopg2.connect(user=username,password=password,host=host,port=port,database=database)
        cursor = connection.cursor()
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
        #CEK APAKAH TABLE SUDAH TERSEDIA 
        cursor.execute("SELECT * FROM information_schema.tables where table_name=%s", (table,))
        exist = bool(cursor.rowcount)
        #JIKA SUDAH TERSEDIA AKAN DIHAPUS DAN DI BUAT ULANG
        if exist == True:
            cursor.execute("DROP TABLE "+ table + " CASCADE")
            cursor.execute("CREATE TABLE "+table+" (index SERIAL, news_id varchar, news_code varchar, news_link varchar, news_date date, news_title varchar, news_content varchar, keyword varchar, wilayah varchar);")
        #JIKA TIDAK TERSEDIA AKAN DIBUAT
        else:
            cursor.execute("CREATE TABLE "+table+" (index SERIAL, news_id varchar, news_code varchar, news_link varchar, news_date date, news_title varchar, news_content varchar, keyword varchar, wilayah varchar);")
        #MEMASUKAN DATA KE DATABASE
        cursor.execute('INSERT INTO '+table+'(news_content, news_id, news_code, news_link, news_date, news_title, keyword, wilayah) values ' +rawstr)
        
        #MELEMPAR TOTAL DATA YANG BERHASIL DIMASUKAN
        return len(arraycontent)  
        
    #GAGAL KONEKSI
    except (Exception, psycopg2.Error) as error :
        return error
    
    #TUTUP KONEKSI
    finally:
        if(connection):
            cursor.close()
            connection.close()

In [25]:
#DEFINISIKAN DATA DATA DARI SINI

In [26]:
#DEFINISIKAN COLUMN DISINI
columns = [
    "news_id",
    "news_code",
    "news_type",
    "news_link",
    "media_displayName",
    "news_date",
    "news_title",
    "news_content",
    "news_image",
    "news_urlimage",
    "news_created_date",
]

#VARIABLE KONEKSI KE DATABASE
host = "localhost"
username = "postgres"
password = "1234567890"
port = "5432"
database = "imm"
table = 'newsimm'
#MENGAMBIL FILE DARI DIREKTORY
filePath = 'C:/Users/ASUS/Documents/bappenas/data mentah/IMM/news.csv';
#KONEKSI KE DATABASE
engine = create_engine('postgresql://'+username+':'+password+'@'+host+':'+port+'/'+database)

#CEK KONEKSI DAN MENYIMPAN DATA KE DATABASE MELALUI FUNGSI UPLOADTOPSQL
checkUpload = uploadToPSQL(columns, table, filePath, engine)
#JIKA BERHASIL AKAN MUNCUL ALERT SUCCES, JIKA GAGAL AKAN MUNCUL ALERT ERROR
if checkUpload != False:
    print("Success, Total Data : " + str(checkUpload))
else:
    print("Error When Upload CSV")

Success, Total Data : 149
